# Mathematical Program MultibodyPlant Tutorial
For instructions on how to run these tutorial notebooks, please see the [index](./index.ipynb).


This shows examples of:
* Creating a `MultibodyPlant` containing an IIWA arm
* Solve a simple inverse kinematics problem by writing a custom evaluator
for `MathematicalProgram` that can handle both `float` and `AutoDiffXd`
inputs
* Using the custom evaluator in a constraint
* Using the custom evaluator in a cost.

***To be added***:
* Using `pydrake.multibody.inverse_kinematics`.
* Visualizing with Meshcat.

### Important Note

Please review the
[API for `pydrake.multibody.inverse_kinematics`](
https://drake.mit.edu/pydrake/pydrake.multibody.inverse_kinematics.html)
before you delve too far into writing custom evaluators for use with
`MultibodyPlant`. You may find the functionality you want there.

## Inverse Kinematics Problem

In this tutorial, we will be solving a simple inverse kinematics problem to
put Link 7's origin at a given distance from a target position. We will use
`MathematicalProgram` to solve this problem in two different ways: first
using the evaluator as a constraint (with a minimum and maximum distance),
and second using the evaluator as a cost (to get as close as possible).

For more information about `MathematicalProgram`, please see the
[`MathematicalProgram` Tutorial](./mathematical_program.ipynb).

## Setup

First, we will import the necessary modules and load a `MultibodyPlant`
containing an IIWA.

In [1]:
import numpy as np
import pydrake
from pydrake.math import RigidTransform
from pydrake.multibody.parsing import Parser
from pydrake.systems.analysis import Simulator
from pydrake.all import MultibodyPlant
from copy import deepcopy
from pydrake.multibody.all import JacobianWrtVariable
from pydrake.autodiffutils import AutoDiffXd
from pydrake.solvers import MathematicalProgram, Solve

In [2]:
plant_f = MultibodyPlant(0.0)
(planar_walker,) = Parser(plant_f).AddModels("/home/anirudhkailaje/Documents/01_UPenn/02_MEAM5170/05_Madness/src/planar_walker.urdf")

# Define some short aliases for frames.
W = plant_f.world_frame()
base = plant_f.GetFrameByName("base", planar_walker)
Left_foot = plant_f.GetFrameByName("left_lower_leg", planar_walker)
Right_foot = plant_f.GetFrameByName("right_lower_leg", planar_walker)

plant_f.WeldFrames(W, base)
plant_f.Finalize()

# Allocate float context to be used by evaluators.
context_f = plant_f.CreateDefaultContext()
# Create AutoDiffXd plant and corresponding context.
plant_ad = plant_f.ToAutoDiffXd() #Autodiff plant
context_ad = plant_ad.CreateDefaultContext()

n_q = plant_ad.num_positions(); n_v = plant_ad.num_velocities(); n_x = n_q+n_v; n_u = plant_ad.num_actuators()
q = np.zeros(n_x); q[0] = 0; q[1] = 0.8
theta = -np.arccos(q[1])
q[3] = theta; q[4] = -2 * theta
q[5] = theta;   q[6] = -2 * theta
initial_state =q
num_knot_points = 5
jump_height = 1.3
jump_height_tol = 1e-2
jump_time = 3 #Alloted time to get airborne

prog = MathematicalProgram()
x = np.array([prog.NewContinuousVariables(n_x, f"x_{i}") for i in range(num_knot_points)])
u = np.array([prog.NewContinuousVariables(n_u, f"u_{i}") for i in range(num_knot_points)])
lambda_c = np.array([prog.NewContinuousVariables(6, f"lambda_{i}") for i in range(num_knot_points)])
lambda_col = np.array([prog.NewContinuousVariables(6, f"collocationlambda_{i}") for i in range(num_knot_points-1)])
timesteps = np.linspace(0.0, jump_time, num_knot_points)
dt = timesteps[1]-timesteps[0]

x0 = x[0]; xf = x[-1]
"""Initial State Constraint"""
prog.AddLinearEqualityConstraint(x0, initial_state)

In [3]:
def EvaluateDynamics(plant, context, state, control, contact_force):
    AD_plant_local = plant; AD_plant_context = context
    AD_plant_local.SetPositionsAndVelocities(AD_plant_context, state)

    M = AD_plant_local.CalcMassMatrixViaInverseDynamics(AD_plant_context)
    B = AD_plant_local.MakeActuationMatrix()
    g = AD_plant_local.CalcGravityGeneralizedForces(AD_plant_context)
    C = AD_plant_local.CalcBiasTerm(AD_plant_context)

    Jleft = AD_plant_local.CalcJacobianTranslationalVelocity(AD_plant_context, JacobianWrtVariable.kV, 
                                                            AD_plant_local.GetFrameByName("left_lower_leg"), np.array([0,0, -0.5]), 
                                                            AD_plant_local.world_frame())
    Jright = AD_plant_local.CalcJacobianTranslationalVelocity(AD_plant_context, JacobianWrtVariable.kV, 
                                                            AD_plant_local.GetFrameByName("right_lower_leg"), np.array([0,0, -0.5]), 
                                                            AD_plant_local.world_frame())
    JdotVleft = AD_plant_local.CalcBiasTranslationalAcceleration(AD_plant_context, JacobianWrtVariable.kV,
                                                            AD_plant_local.GetFrameByName("left_lower_leg"), np.array([0,0,-0.5]),
                                                            AD_plant_local.world_frame())
    JdotVright = AD_plant_local.CalcBiasTranslationalAcceleration(AD_plant_context, JacobianWrtVariable.kV,
                                                            AD_plant_local.GetFrameByName("right_lower_leg"), np.array([0,0,-0.5]),
                                                            AD_plant_local.world_frame())
    
    J_c = np.row_stack((Jleft, Jright)); J_c_dot_v = np.row_stack((JdotVleft, JdotVright))

    if (state.dtype == AutoDiffXd):
        M_inv = pydrake.math.inv(state)
    else:
        M_inv = np.linalg.inv(M)
    v_dot = M_inv @(B@control+g-C+J_c.T@contact_force)
    x_dot = np.hstack((state[-AD_plant_local.num_velocities():], v_dot))
    foot_acceleration = (J_c @ v_dot).reshape(-1, 1) + J_c_dot_v

    left_foot_zpos = AD_plant_local.CalcPointsPositions(AD_plant_context, 
                                                        AD_plant_local.GetFrameByName("left_lower_leg"), np.array([0,0,-0.5]), 
                                                        AD_plant_local.world_frame())[2]
    right_foot_zpos = AD_plant_local.CalcPointsPositions(AD_plant_context, 
                                                         AD_plant_local.GetFrameByName("right_lower_leg"), np.array([0,0,-0.5]), 
                                                         AD_plant_local.world_frame())[2]
    foot_zpos = np.row_stack((left_foot_zpos, right_foot_zpos))

    return x_dot, foot_acceleration, foot_zpos

def CollocationConstraintEvaluator(AD_plant_local, AD_plant_context, state_i, state_i1, control_i, control_i1, lambda_i, lambda_i1, lambda_col, dt):
        
    fi,footacc_i, footpos_i = EvaluateDynamics(AD_plant_local, AD_plant_context, state_i, control_i, lambda_i)
    fi1, footacc_i1, footpos_i1 = EvaluateDynamics(AD_plant_local, AD_plant_context, state_i1, control_i1, lambda_i1)

    collocation_state = 0.5*(state_i+state_i1) - 0.125*dt*(fi1-fi)
    collocation_control = 0.5*(control_i+control_i1)
    collocation_state_dot = 1.5*(state_i1-state_i)/dt -0.25*(fi+fi1)

    fcol, footacc_col, footpos_col = EvaluateDynamics(AD_plant_local, AD_plant_context, collocation_state, collocation_control, lambda_col)
    h_i = collocation_state_dot-fcol
    colloction_constraints_params = np.hstack(([h_i, footacc_i, footacc_col, footpos_i, footpos_col]))
    last_point_constraints_params = np.hstack([footacc_i1, footpos_i1])
    return colloction_constraints_params, last_point_constraints_params


In [4]:

"""Dynamics constraints through collocation constraints
For each collocation point, x_dot = 0 (Satisfy Dynamics), foot_acceleration = 0, foot_pos = [0,0]
"""
x_dot_lb = np.zeros(n_x); x_dot_ub = x_dot_lb; x_dot_eps = 1e-4
footacc_lb = np.zeros(6); footacc_ub = footacc_lb; footacc_eps =1e-4
footpos_lb = np.zeros(2); footpos_eps = 1e-3; footpos_ub = footpos_lb+footpos_eps

def AddCollocationConstraints(program, N, X_var, U_var, lambda_var, lambda_col_var, timesteps):
    prog = program
    if X_var[0].dtype == float:
        AD_plant_local = plant_f
        AD_plant_context = context_f
    else:
        AD_plant_local = plant_ad
        AD_plant_context = context_ad
    n_x = AD_plant_local.num_positions()+AD_plant_local.num_velocities(); n_u = AD_plant_local.num_actuators()
    dt = timesteps[1]-timesteps[0]
    for i in range(N-1):
        #Sorry for the sin, tried literally every other way, I'm dissapointed in life.
        
        def CollocationConstraintHelper(vars):
            state_i = vars[:n_x]; state_i1 = vars[n_x:2*n_x]
            u_i = vars[2*n_x:2*n_x+n_u]; u_i1 = vars[2*n_x+n_u: 2*(n_x+n_u)]
            lambda_i = vars[2*(n_x+n_u):2*(n_x+n_u)+6]; lambda_i1 = vars[2*(n_x+n_u)+6: 2*(n_x+n_u)+12]; collocation_lambda = vars[2*(n_x+n_u)+12:]
            return CollocationConstraintEvaluator(AD_plant_local, AD_plant_context, state_i, state_i1, u_i, u_i1, lambda_i, lambda_i1, collocation_lambda, dt)[0]
        
        def LastPointConstraintHelper(vars):
            state_i = vars[:n_x]; state_i1 = vars[n_x:2*n_x]
            u_i = vars[2*n_x:2*n_x+n_u]; u_i1 = vars[2*n_x+n_u: 2*(n_x+n_u)]
            lambda_i = vars[2*(n_x+n_u):2*(n_x+n_u)+6]; lambda_i1 = vars[2*(n_x+n_u)+6: 2*(n_x+n_u)+12]; collocation_lambda = vars[2*(n_x+n_u)+12:]
            return CollocationConstraintEvaluator(AD_plant_local, AD_plant_context, state_i, state_i1, u_i, u_i1, lambda_i, lambda_i1, collocation_lambda, dt)[1]
        
        collocation_lb = np.hstack(([x_dot_lb, footacc_lb, footacc_lb, footpos_lb,footpos_lb]))
        collocation_ub = np.hstack(([x_dot_ub, footacc_ub, footacc_ub, footpos_ub, footpos_ub]))
        lastpoint_lb = np.hstack(([footacc_lb, footpos_lb]))
        lastpoint_ub = np.hstack(([footacc_ub, footpos_ub]))
        variables = np.hstack(([X_var[i], X_var[i+1], U_var[i], U_var[i+1], lambda_var[i], lambda_var[i+1], lambda_col_var[i]]))
        prog.AddConstraint(CollocationConstraintHelper, collocation_lb, collocation_ub, variables)
        
        
        if i == num_knot_points-1:
            prog.AddConstraint(LastPointConstraintHelper, lastpoint_lb, lastpoint_ub, variables)

   
AddCollocationConstraints(prog, num_knot_points, x, u, lambda_c, lambda_col, timesteps)

In [5]:
"""Adding Cost on Controls"""
for i in range(num_knot_points-1):
    prog.AddQuadraticCost(0.5*(timesteps[i+1]-timesteps[i])*(u[i].T@u[i])+(u[i+1].T@u[i+1]))

In [6]:
prog.GetAllConstraints()

In [7]:
prog.GetAllCosts()

In [8]:
prog.SetInitialGuess(x, np.zeros((num_knot_points, n_x)))
prog.SetInitialGuess(u, np.zeros((num_knot_points, n_u)))

In [12]:
x[0].shape

(14,)

In [10]:
pydrake.autodiffutils.ExtractValue(x)

TypeError: ExtractValue(): incompatible function arguments. The following argument types are supported:
    1. (auto_diff_matrix: numpy.ndarray[object[m, n]]) -> numpy.ndarray[numpy.float64[m, n]]

Invoked with: array([[Variable('x_0(0)', Continuous), Variable('x_0(1)', Continuous),
        Variable('x_0(2)', Continuous), Variable('x_0(3)', Continuous),
        Variable('x_0(4)', Continuous), Variable('x_0(5)', Continuous),
        Variable('x_0(6)', Continuous), Variable('x_0(7)', Continuous),
        Variable('x_0(8)', Continuous), Variable('x_0(9)', Continuous),
        Variable('x_0(10)', Continuous), Variable('x_0(11)', Continuous),
        Variable('x_0(12)', Continuous), Variable('x_0(13)', Continuous)],
       [Variable('x_1(0)', Continuous), Variable('x_1(1)', Continuous),
        Variable('x_1(2)', Continuous), Variable('x_1(3)', Continuous),
        Variable('x_1(4)', Continuous), Variable('x_1(5)', Continuous),
        Variable('x_1(6)', Continuous), Variable('x_1(7)', Cont

In [11]:
result = Solve(prog)

print(f"Success? {result.is_success()}")
print(result.get_solution_result())

RuntimeError: Exception while evaluating SNOPT costs and constraints: 'TypeError: CalcJacobianTranslationalVelocity(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.multibody.plant.MultibodyPlant_𝓣AutoDiffXd𝓤, context: pydrake.systems.framework.Context_𝓣AutoDiffXd𝓤, with_respect_to: pydrake.multibody.tree.JacobianWrtVariable, frame_B: pydrake.multibody.tree.Frame_𝓣AutoDiffXd𝓤, p_BoBi_B: numpy.ndarray[object[3, n], flags.f_contiguous], frame_A: pydrake.multibody.tree.Frame_𝓣AutoDiffXd𝓤, frame_E: pydrake.multibody.tree.Frame_𝓣AutoDiffXd𝓤) -> numpy.ndarray[object[m, n]]

Invoked with: <pydrake.multibody.plant.MultibodyPlant_𝓣AutoDiffXd𝓤 object at 0x7f23e57987f0>, <pydrake.systems.framework.LeafContext_𝓣AutoDiffXd𝓤 object at 0x7f23e15e6d70>, <JacobianWrtVariable.kV: 1>, <BodyFrame_[AutoDiffXd] name='left_lower_leg' index=8 model_instance=2>, array([ 0. ,  0. , -0.5]), <BodyFrame_[AutoDiffXd] name='world' index=0 model_instance=0>

At:
  

In [ ]:
# Allocate float context to be used by evaluators.
context_f = plant_f.CreateDefaultContext()
# Create AutoDiffXd plant and corresponding context.
plant_ad = plant_f.ToAutoDiffXd()
context_ad = plant_ad.CreateDefaultContext()

def resolve_frame(plant, F):
    """Gets a frame from a plant whose scalar type may be different."""
    return plant.GetFrameByName(F.name(), F.model_instance())

# Define target position.
p_WT = [0, 0, 0]

def link_7_distance_to_target(q):
    """Evaluates squared distance between L7 origin and target T."""
    # Choose plant and context based on dtype.
    if q.dtype == float:
        plant = plant_f
        context = context_f
    else:
        # Assume AutoDiff.
        plant = plant_ad
        context = context_ad
    # Do forward kinematics.
    plant.SetPositions(context, planar_walker, q)
    X_WL7 = plant.CalcRelativeTransform(
        context, resolve_frame(plant, W), resolve_frame(plant, Left_foot))
    p_TL7 = X_WL7.translation() - p_WT
    return p_TL7.dot(p_TL7)

# WARNING: If you return a scalar for a constraint, or a vector for
# a cost, you may get the following cryptic error:
# "Unable to cast Python instance to C++ type"
link_7_distance_to_target_vector = lambda q: [link_7_distance_to_target(q)]

## Formulating the Optimization Problems

### Formluation 1: Using the Custom Evaluator in a Constraint

We will formulate and solve the problem with a basic cost and our custom
evaluator in a constraint.

Note that we use the vectorized version of the evaluator.

In [ ]:
prog = MathematicalProgram()

q = prog.NewContinuousVariables(plant_f.num_positions())
# Define nominal configuration.
q0 = np.zeros(plant_f.num_positions())

# Add basic cost. (This will be parsed into a QuadraticCost.)
prog.AddCost((q - q0).dot(q - q0))

# Add constraint based on custom evaluator.
prog.AddConstraint(
    link_7_distance_to_target_vector,
    lb=[0.1], ub=[0.2], vars=q)

In [ ]:
result = Solve(prog, initial_guess=q0)

print(f"Success? {result.is_success()}")
print(result.get_solution_result())
q_sol = result.GetSolution(q)
print(q_sol)

print(f"Initial distance: {link_7_distance_to_target(q0):.3f}")
print(f"Solution distance: {link_7_distance_to_target(q_sol):.3f}")

In [ ]:
q = np.zeros((plant_ad.num_positions()+plant_ad.num_velocities(),))
q[0] = 0; q[1] = 0.8
theta = -np.arccos(q[1])
q[3] = theta; q[4] = -2 * theta
q[5] = theta;   q[6] = -2 * theta

In [ ]:
q = prog.NewContinuousVariables(14, "q")

In [ ]:
plant_ad.SetPositionsAndVelocities(context_ad, q)

In [ ]:
prog.SetInitialGuess(x, q)

In [ ]:
pydrake.autodiffutils.ExtractValue()

In [ ]:
x[0]

In [ ]:
plant_ad.SetPositionsAndVelocities(context_ad, x[0])

In [ ]:
plant_ad.GetPositions(context_ad)

In [ ]:
plant_ad.CalcPointsPositions(context_ad, plant_ad.GetFrameByName("left_lower_leg"), np.array([0,0,-0.5]), plant_ad.world_frame())

In [ ]:
plant_ad.CalcPointsPositions(context_ad, plant_ad.GetFrameByName("right_lower_leg"), np.array([0,0,-0.5]), plant_ad.world_frame())

### Formulation 2: Using Custom Evaluator in a Cost

We will formulate and solve the problem, but this time we will use our custom
evaluator in a cost.

Note that we use the scalar version of the evaluator.

In [ ]:
prog = MathematicalProgram()

q = prog.NewContinuousVariables(plant_f.num_positions())
# Define nominal configuration.
q0 = np.zeros(plant_f.num_positions())

# Add custom cost.
prog.AddCost(link_7_distance_to_target, vars=q)

In [ ]:
result = Solve(prog, initial_guess=q0)

print(f"Success? {result.is_success()}")
print(result.get_solution_result())
q_sol = result.GetSolution(q)
print(q_sol)

print(f"Initial distance: {link_7_distance_to_target(q0):.3f}")
print(f"Solution distance: {link_7_distance_to_target(q_sol):.3f}")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2b4fc509-aef2-417d-a40d-6071dfed9199' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>